## Imports

In [1]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Carregamento de Dados
---

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

In [7]:
spark

### Montando o drive

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Carregando os dados dos Resultados

In [9]:
path = '/content/drive/MyDrive/resultados/SP_turno_1.csv'
resultados = spark.read.csv(path, sep=';', inferSchema=True, header=True, encoding='ISO-8859-1')

### Carregando os dados do Eleitorado

In [10]:
path = '/content/drive/MyDrive/eleitorado/perfil_eleitorado_2020/perfil_eleitorado_2020.csv'
eleitorado = spark.read.csv(path, sep=';', inferSchema=True, header=True, encoding='ISO-8859-1')

# Manipulando os Dados
---

## Operações básicas

In [13]:
resultados.limit(5).toPandas()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_PLEITO,DT_PLEITO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,CD_CARGO_PERGUNTA,DS_CARGO_PERGUNTA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,DT_BU_RECEBIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,CD_TIPO_URNA,DS_TIPO_URNA,CD_TIPO_VOTAVEL,DS_TIPO_VOTAVEL,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_URNA_EFETIVADA,CD_CARGA_1_URNA_EFETIVADA,CD_CARGA_2_URNA_EFETIVADA,CD_FLASHCARD_URNA_EFETIVADA,DT_CARGA_URNA_EFETIVADA,DS_CARGO_PERGUNTA_SECAO,DS_AGREGADAS,DT_ABERTURA,DT_ENCERRAMENTO,QT_ELEITORES_BIOMETRIA_NH,DT_EMISSAO_BU,NR_JUNTA_APURADORA,NR_TURMA_APURADORA
0,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,SP,71072,SÃO PAULO,1,1,1015,11,Prefeito,-1,#NULO#,#NULO#,15/11/2020 22:26:42,386,241,145,1,APURADA,2,Branco,95,Branco,5,1618073,432.454.548.163.181.015.,306.728,E0BA7264,03/11/2020 11:19:00,11 - 1,#NULO#,15/11/2020 07:00:00,15/11/2020 17:02:46,0,15/11/2020 17:04:06,-1,-1
1,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,SP,71072,SÃO PAULO,1,1,1015,11,Prefeito,13,PT,Partido dos Trabalhadores,15/11/2020 22:26:42,386,241,145,1,APURADA,1,Nominal,13,JILMAR TATTO,4,1618073,432.454.548.163.181.015.,306.728,E0BA7264,03/11/2020 11:19:00,11 - 1,#NULO#,15/11/2020 07:00:00,15/11/2020 17:02:46,0,15/11/2020 17:04:06,-1,-1
2,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,SP,71072,SÃO PAULO,1,1,1015,11,Prefeito,18,REDE,Rede Sustentabilidade,15/11/2020 22:26:42,386,241,145,1,APURADA,1,Nominal,18,MARINA HELOU,1,1618073,432.454.548.163.181.015.,306.728,E0BA7264,03/11/2020 11:19:00,11 - 1,#NULO#,15/11/2020 07:00:00,15/11/2020 17:02:46,0,15/11/2020 17:04:06,-1,-1
3,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,SP,71072,SÃO PAULO,1,1,1015,11,Prefeito,40,PSB,Partido Socialista Brasileiro,15/11/2020 22:26:42,386,241,145,1,APURADA,1,Nominal,40,MÁRCIO FRANÇA,29,1618073,432.454.548.163.181.015.,306.728,E0BA7264,03/11/2020 11:19:00,11 - 1,#NULO#,15/11/2020 07:00:00,15/11/2020 17:02:46,0,15/11/2020 17:04:06,-1,-1
4,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,SP,71072,SÃO PAULO,1,1,1015,11,Prefeito,45,PSDB,Partido da Social Democracia Brasileira,15/11/2020 22:26:42,386,241,145,1,APURADA,1,Nominal,45,BRUNO COVAS,111,1618073,432.454.548.163.181.015.,306.728,E0BA7264,03/11/2020 11:19:00,11 - 1,#NULO#,15/11/2020 07:00:00,15/11/2020 17:02:46,0,15/11/2020 17:04:06,-1,-1


In [14]:
eleitorado.limit(5).toPandas()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,CD_MUN_SIT_BIOMETRIA,DS_MUN_SIT_BIOMETRIA,NR_ZONA,CD_GENERO,DS_GENERO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
0,01/08/2020,08:37:48,2020,SP,70734,SÃO BENTO DO SAPUCAÍ,0,Sem biometria,314,4,FEMININO,3,CASADO,7074,70 a 74 anos,3,ENSINO FUNDAMENTAL INCOMPLETO,47,37,1,0
1,01/08/2020,08:37:48,2020,SP,70734,SÃO BENTO DO SAPUCAÍ,0,Sem biometria,314,4,FEMININO,3,CASADO,7074,70 a 74 anos,4,ENSINO FUNDAMENTAL COMPLETO,5,4,0,0
2,01/08/2020,08:37:48,2020,SP,70734,SÃO BENTO DO SAPUCAÍ,0,Sem biometria,314,4,FEMININO,3,CASADO,7074,70 a 74 anos,5,ENSINO MÉDIO INCOMPLETO,1,1,0,0
3,01/08/2020,08:37:48,2020,SP,70734,SÃO BENTO DO SAPUCAÍ,0,Sem biometria,314,4,FEMININO,3,CASADO,7074,70 a 74 anos,6,ENSINO MÉDIO COMPLETO,10,8,0,0
4,01/08/2020,08:37:48,2020,SP,70734,SÃO BENTO DO SAPUCAÍ,0,Sem biometria,314,4,FEMININO,3,CASADO,7074,70 a 74 anos,8,SUPERIOR COMPLETO,23,18,0,0


### Selecionando as colunas do DataFrame

In [11]:
resultados

DataFrame[DT_GERACAO: string, HH_GERACAO: string, ANO_ELEICAO: int, CD_TIPO_ELEICAO: int, NM_TIPO_ELEICAO: string, CD_PLEITO: int, DT_PLEITO: string, NR_TURNO: int, CD_ELEICAO: int, DS_ELEICAO: string, SG_UF: string, CD_MUNICIPIO: int, NM_MUNICIPIO: string, NR_ZONA: int, NR_SECAO: int, NR_LOCAL_VOTACAO: int, CD_CARGO_PERGUNTA: int, DS_CARGO_PERGUNTA: string, NR_PARTIDO: int, SG_PARTIDO: string, NM_PARTIDO: string, DT_BU_RECEBIDO: string, QT_APTOS: int, QT_COMPARECIMENTO: int, QT_ABSTENCOES: int, CD_TIPO_URNA: int, DS_TIPO_URNA: string, CD_TIPO_VOTAVEL: int, DS_TIPO_VOTAVEL: string, NR_VOTAVEL: int, NM_VOTAVEL: string, QT_VOTOS: int, NR_URNA_EFETIVADA: int, CD_CARGA_1_URNA_EFETIVADA: string, CD_CARGA_2_URNA_EFETIVADA: double, CD_FLASHCARD_URNA_EFETIVADA: string, DT_CARGA_URNA_EFETIVADA: string, DS_CARGO_PERGUNTA_SECAO: string, DS_AGREGADAS: string, DT_ABERTURA: string, DT_ENCERRAMENTO: string, QT_ELEITORES_BIOMETRIA_NH: int, DT_EMISSAO_BU: string, NR_JUNTA_APURADORA: int, NR_TURMA_APURA

In [12]:
eleitorado

DataFrame[DT_GERACAO: string, HH_GERACAO: string, ANO_ELEICAO: int, SG_UF: string, CD_MUNICIPIO: int, NM_MUNICIPIO: string, CD_MUN_SIT_BIOMETRIA: int, DS_MUN_SIT_BIOMETRIA: string, NR_ZONA: int, CD_GENERO: int, DS_GENERO: string, CD_ESTADO_CIVIL: int, DS_ESTADO_CIVIL: string, CD_FAIXA_ETARIA: int, DS_FAIXA_ETARIA: string, CD_GRAU_ESCOLARIDADE: int, DS_GRAU_ESCOLARIDADE: string, QT_ELEITORES_PERFIL: int, QT_ELEITORES_BIOMETRIA: int, QT_ELEITORES_DEFICIENCIA: int, QT_ELEITORES_INC_NM_SOCIAL: int]

In [13]:
resultados = resultados.select("NM_MUNICIPIO", "NR_ZONA", "NR_SECAO", "DS_CARGO_PERGUNTA", "SG_PARTIDO", "QT_APTOS", "QT_COMPARECIMENTO", "QT_ABSTENCOES", "NM_VOTAVEL", "QT_VOTOS")

In [14]:
eleitorado = eleitorado.select("NM_MUNICIPIO", "NR_ZONA", "DS_GENERO", "DS_ESTADO_CIVIL", "DS_FAIXA_ETARIA", "DS_GRAU_ESCOLARIDADE", "QT_ELEITORES_PERFIL", "QT_ELEITORES_DEFICIENCIA", "QT_ELEITORES_INC_NM_SOCIAL")

## Encontrando o prefeito mais votado de cada cidade

In [128]:
# Aqui, estamos agrupando os resultados com base em múltiplas colunas:
# Para cada grupo, estamos somando os votos ("QT_VOTOS") e renomeando essa soma para "TOTAL_VOTOS".
candidato_mais_votado = resultados.groupBy("NM_MUNICIPIO", "NM_VOTAVEL", "DS_CARGO_PERGUNTA", "SG_PARTIDO").agg(F.sum("QT_VOTOS").alias("TOTAL_VOTOS"))

# Nesta linha, estamos filtrando apenas os registros onde o cargo em disputa é "Prefeito".
candidato_mais_votado_prefeito = candidato_mais_votado.filter(candidato_mais_votado["DS_CARGO_PERGUNTA"] == "Prefeito")

# Aqui, estamos definindo uma janela de análise (window) que particiona os dados pelo "NM_MUNICIPIO" (nome do município)
# e os ordena em ordem decrescente pelo "TOTAL_VOTOS" (quantidade de votos).
window_spec = Window.partitionBy("NM_MUNICIPIO").orderBy(F.desc("TOTAL_VOTOS"))

# Com base na janela definida acima, estamos adicionando uma nova coluna chamada "rank" ao DataFrame.
# Esta coluna "rank" representa o ranking de cada candidato a prefeito em seu respectivo município com base no número total de votos.
candidato_mais_votado_prefeito_ranked = candidato_mais_votado_prefeito.withColumn("rank", F.row_number().over(window_spec))

# Finalmente, filtramos apenas os candidatos que estão em 1º lugar no ranking (ou seja, os mais votados) em cada município.
candidato_mais_votado_in_each_municipio_prefeito = candidato_mais_votado_prefeito_ranked.filter(candidato_mais_votado_prefeito_ranked["rank"] == 1)


In [129]:
candidato_mais_votado_in_each_municipio_prefeito.toPandas()

,NM_MUNICIPIO,NM_VOTAVEL,DS_CARGO_PERGUNTA,SG_PARTIDO,TOTAL_VOTOS,rank
0,CANAS,SILVANA ZANIN,Prefeito,PDT,1108,1
1,PORTO FELIZ,DR CÁSSIO,Prefeito,PTB,25318,1
2,RIBEIRÃO BRANCO,MAURO TEIXEIRA,Prefeito,PP,5914,1
3,SÃO JOAQUIM DA BARRA,SCHMIDT,Prefeito,MDB,9789,1
4,ARCO-ÍRIS,ALDO MANSANO,Prefeito,PV,1134,1
...,...,...,...,...,...,...
640,SÃO SEBASTIÃO DA GRAMA,ZÉ DA DOCA,Prefeito,PTB,2402,1
641,NOVA CANAÃ PAULISTA,THAIS,Prefeito,PSDB,743,1
642,CAMPINA DO MONTE ALEGRE,ZE DITO,Prefeito,PSDB,2439,1
643,SÃO JOSÉ DO BARREIRO,LÊ BRAGA,Prefeito,PSDB,1570,1


## Buscando a porcentagem de votos que um candidato teve em cada zona e informações do perfil de eleitor predominante nela.

In [25]:
def votos_candidato_zona(nome_candidato, partido, df_votos, df_eleitorado):
    # Filtrar o dataframe de votos pelo nome do candidato e partido
    df_candidato_votos = df_votos.filter((df_votos.NM_VOTAVEL == nome_candidato) & (df_votos.SG_PARTIDO == partido))

    # Agrupar pelo NM_MUNICIPIO e NR_ZONA para somar a quantidade de votos por zona
    df_candidato_votos_agrupado = df_candidato_votos.groupBy("NM_MUNICIPIO", "NR_ZONA").agg(F.sum("QT_VOTOS").alias("TOTAL_VOTOS"))

    # Ordenar as zonas pelo número de votos em ordem decrescente
    df_candidato_votos_agrupado = df_candidato_votos_agrupado.orderBy("TOTAL_VOTOS", ascending=False)

    # Juntar o dataframe de eleitorado com o dataframe de votos agrupados usando o NM_MUNICIPIO e NR_ZONA
    df_joined = df_candidato_votos_agrupado.join(df_eleitorado, ["NM_MUNICIPIO", "NR_ZONA"])

    # Computa a soma dos perfis para cada categoria
    df_sum = df_joined.groupBy("NM_MUNICIPIO", "NR_ZONA", "DS_GENERO", "DS_FAIXA_ETARIA", "DS_GRAU_ESCOLARIDADE").agg(F.sum("QT_ELEITORES_PERFIL").alias("TOTAL_PERFIL"))

    # Encontrar a moda das colunas de interesse
    window = Window.partitionBy("NM_MUNICIPIO", "NR_ZONA").orderBy(F.desc("TOTAL_PERFIL"))

    df_genero = df_sum.withColumn("rank", F.row_number().over(window)).filter("rank = 1").select("NM_MUNICIPIO", "NR_ZONA", "DS_GENERO").drop("rank")
    df_faixa = df_sum.withColumn("rank", F.row_number().over(window)).filter("rank = 1").select("NM_MUNICIPIO", "NR_ZONA", "DS_FAIXA_ETARIA").drop("rank")
    df_escolaridade = df_sum.withColumn("rank", F.row_number().over(window)).filter("rank = 1").select("NM_MUNICIPIO", "NR_ZONA", "DS_GRAU_ESCOLARIDADE").drop("rank")

    # Agregar o total de eleitores por NM_MUNICIPIO e NR_ZONA
    df_total_eleitores = df_eleitorado.groupBy("NM_MUNICIPIO", "NR_ZONA").agg(F.sum("QT_ELEITORES_PERFIL").alias("TOTAL_ELEITORES"))

    # Calcular a porcentagem de votos
    df_candidato_votos_agrupado = df_candidato_votos_agrupado.join(df_total_eleitores, ["NM_MUNICIPIO", "NR_ZONA"])
    df_candidato_votos_agrupado = df_candidato_votos_agrupado.withColumn("PERCENTUAL_VOTOS", F.round((F.col("TOTAL_VOTOS") / F.col("TOTAL_ELEITORES")) * 100, 2))
    # Junta os resultados
    df_result = df_candidato_votos_agrupado.join(df_genero, ["NM_MUNICIPIO", "NR_ZONA"]).join(df_faixa, ["NM_MUNICIPIO", "NR_ZONA"]).join(df_escolaridade, ["NM_MUNICIPIO", "NR_ZONA"])

    return df_result

In [34]:
result = votos_candidato_zona('ROGÉRIO LINS', 'PODE', resultados, eleitorado)
result_pandas = result.toPandas()

In [35]:
result_pandas

,NM_MUNICIPIO,NR_ZONA,TOTAL_VOTOS,TOTAL_ELEITORES,PERCENTUAL_VOTOS,DS_GENERO,DS_FAIXA_ETARIA,DS_GRAU_ESCOLARIDADE
0,OSASCO,331,41073,120639,34.05,FEMININO,35 a 39 anos,ENSINO MÉDIO COMPLETO
1,OSASCO,332,28002,72494,38.63,FEMININO,35 a 39 anos,ENSINO MÉDIO COMPLETO
2,OSASCO,277,27758,79878,34.75,FEMININO,35 a 39 anos,ENSINO MÉDIO COMPLETO
3,OSASCO,315,46134,127556,36.17,FEMININO,35 a 39 anos,ENSINO MÉDIO COMPLETO
4,OSASCO,213,26213,73770,35.53,FEMININO,35 a 39 anos,SUPERIOR COMPLETO
5,OSASCO,276,35027,93024,37.65,FEMININO,35 a 39 anos,ENSINO MÉDIO COMPLETO


In [36]:
result_pandas.to_csv('ResultadosRLPODE2020.csv', sep=';', encoding='ISO-8859-1')